# Reconstructing the Drosophila embryo
This tutorial demonstrates how to use `novoSpaRc` to reconstruct single-cell gene expression by using the Drosophila scRNA-seq dataset [Karaiskos et al., 2017, Science]. The two modes of `novoSpaRc` are described: (1) de novo, without any prior information and (2) with using a set of gene expression markers.

In [19]:
import numpy as np
import novosparc
import matplotlib
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
import matplotlib
matplotlib.use('TKAgg',warn=False, force=True)

## Import data
Import the gene names and the gene expression matrix. Although `novoSpaRc` can also operate on raw data, it performs much better when the gene expression matrix is already normalized. Note the format: cells in rows and genes in columns.

In [20]:
data_path = 'novosparc/datasets/drosophila_scRNAseq/dge_normalized.txt'
dataset = novosparc.io.load_data(data_path).T

Display some information about the dataset

In [21]:
gene_names = np.asarray(dataset.var.index.tolist())
dge = dataset.X
print ( gene_names)
print ('total number of genes in the data:', len(gene_names), '\n')
print (dge)
print ('number of cells and genes in the matrix:', dge.shape)

['128up' '140up' '14-3-3epsilon' ... 'zyd' 'zye' 'Zyx']
total number of genes in the data: 8924 

[[2.57  1.16  8.838 ... 1.16  0.    3.443]
 [2.509 3.007 8.788 ... 1.742 0.    4.305]
 [0.    0.    8.843 ... 0.    0.    4.082]
 ...
 [1.972 2.775 8.333 ... 0.    0.    3.665]
 [0.    0.    8.554 ... 0.    0.    5.354]
 [0.    0.    7.78  ... 0.    0.    3.962]]
number of cells and genes in the matrix: (1297, 8924)


It is advisable to subset the gene expression matrix to the highly variable genes. Following reads a file with the list of highly variable genes to subset the dataset. Alternatively, scanpy highly variable gene functions can be used directly on the dataset to calculate the highly variable genes.

In [22]:
dataset.raw = dataset # this stores the current dataset with all the genes for future use

hvg_path = 'novosparc/datasets/drosophila_scRNAseq/high_var_genes.txt'
dataset, hvg = novosparc.pp.subset_to_hvg(dataset, hvg_file = hvg_path)

print ('number of highly variable genes:', len(hvg))
print ('number of cells and genes in the matrix:', dataset.X.shape)

number of highly variable genes: 476
number of cells and genes in the matrix: (1297, 476)


### Optional: subsample the gene expression matrix
Subsample the matrix to a smaller number of cells. This helps to first setup the whole reconstruction script quickly and to later assess robustness of the different spatial reconstructions. 

In [23]:
#cells_selected, dataset = novosparc.pp.subsample_dataset(dataset, 50)

## Create the target space
Creating the target space that represents the biological tissue plays an important role, as it influences the geometric component of the reconstruction. In the case where the physical cellular locations are known, such as in the case of the early Drosophila embryo, it is better to use these:

In [24]:
target_space_path = 'novosparc/datasets/bdtnp/geometry.txt'
locations = novosparc.io.load_target_space(target_space_path, is_2D=True)
locations = locations[np.random.choice(locations.shape[0], 3039), :]

TypeError: load_target_space() got an unexpected keyword argument 'is_2D'

In [ ]:
print ('dimensions of tissue space:', locations.shape, '\n')
print (locations)

In [ ]:
%matplotlib inline
plt.figure()
plt.scatter(locations[:, 0], locations[:, 1], s=1)
plt.show()
plt.close()

### Alternative tissue spaces
If the cellular locations are not known, the user needs to create. `novoSpaRc` has currently two functions that enable a quick construction of target spaces.
#### Map to a rectangular grid
The function `construct_target_grid` can be used to construct a basic rectangular grid as target space. The only parameter is the total number of cellular locations on the target grid.

In [25]:
locations = novosparc.geometry.construct_target_grid(500)
plt.figure()
plt.scatter(locations[:, 0], locations[:, 1], s=1)
plt.show()
plt.close()

#### Map to a user-generated space
The function `create_target_space_from_image` creates a target space by using a user-generated image. The image has to be a black shape drawn onto a white background, as the one below:

<div>
<img src="https://raw.githubusercontent.com/nukappa/nukappa.github.io/master/images/tissue_example.png", width="250"/>
</div>

In [26]:
tissue_path = 'novosparc/datasets/tissue_example.png'
locations = novosparc.geometry.create_target_space_from_image(tissue_path)

This will probably create a very large number of locations, so it's better to subsample them.

In [27]:
print ('dimension of original tissue space:', locations.shape)
locations = locations[np.random.choice(locations.shape[0], 3039), :]
num_locations = locations.shape[0]
print ('dimension of downsampled tissue space:', locations.shape)
plt.figure()
plt.scatter(locations[:, 0], locations[:, 1], s=1)
plt.show()
plt.close()

dimension of original tissue space: (125058, 2)
dimension of downsampled tissue space: (3039, 2)


## Reconstruct the tissue de novo
We first prepare for the reconstruction by computing the cost matrices. If the tissue space is 1-dimensional (e.g. a line), `num_neighbors` can be reduced to 3.

We first create a Tissue object. This is initiated by using the single cell dataset, location coordinates and the output folder to save the results.

In [28]:
tissue = novosparc.cm.Tissue(dataset=dataset, locations=locations)

Then, we prepare for the reconstruction by computing the cost matrices. If the tissue space is 1-dimensional (e.g a line), num_neighbours can be reduced to 3.


In [29]:
tissue.setup_reconstruction(num_neighbors_s = 5, num_neighbors_t = 5)

Now we can reconstruct our tissue. The `alpha_linear` parameter controls the contribution of the two branches in `novoSpaRc`: `alpha_linear=0` means that no prior information is available, while `alpha_linear=1` implies that the reconstruction will be only based on the reference marker genes provided. Here we aim for a de novo reconstruction.

In [30]:
tissue.reconstruct(alpha_linear=0, epsilon=5e-3)

Reconstructing spatial information with 0 markers: 1297 cells and 3039 locations ... 
It.  |Err         
-------------------
    0|2.305119e-18|


This is the final object: the sDGE (spatial gene expression matrix) holds expression values for the highly variable genes across all locations.

In [31]:
sdge_hvg = tissue.sdge

The reconstruction is based on the highly variable genes only, but the spatial expression of the rest can also be recovered

In [32]:
print(sdge_hvg.shape)

(476, 3039)


In [33]:
sdge = tissue.calculate_sdge_for_all_genes()
print ('dimensions of the sDGE (genes, locations):', sdge.shape)

dimensions of the sDGE (genes, locations): (8924, 3039)


### Plot spatial gene expression
Plot gene expression patterns in space. In this case the de novo reconstruction is not enough to recover the expression pattern of `ftz` from the data.

In [41]:
plt.figure(figsize=(12,4))
plt.subplot(231)
plt.scatter(locations[:, 0], locations[:, 1], c=sdge[np.argwhere(gene_names == 'ftz')[0], :].flatten(), s=1)
plt.axis('off')
plt.subplot(232)
plt.scatter(locations[:, 0], locations[:, 1], c=sdge[np.argwhere(gene_names == 'twi')[0], :].flatten(), s=1)
plt.axis('off')
plt.subplot(233)
plt.scatter(locations[:, 0], locations[:, 1], c=sdge[np.argwhere(gene_names == 'zen')[0], :].flatten(), s=1)
plt.axis('off')
plt.subplot(234)
plt.scatter(locations[:, 0], locations[:, 1], c=sdge[np.argwhere(gene_names == 'ImpE2')[0], :].flatten(), s=1)
plt.axis('off')
plt.subplot(235)
plt.scatter(locations[:, 0], locations[:, 1], c=sdge[np.argwhere(gene_names == 'sim')[0], :].flatten(), s=1)
plt.axis('off')
plt.subplot(236)
plt.scatter(locations[:, 0], locations[:, 1], c=sdge[np.argwhere(gene_names == 'pyr')[0], :].flatten(), s=1)
plt.axis('off')
plt.show()

In [37]:
np.argwhere(gene_names == 'ftz')

array([[5624]])

## Reconstruct the tissue by using reference marker genes
Available reference marker genes assist `novoSpaRc` and result in general in a much better reconstruction. The setup of the reconstruction is the same as before. Here we only need to compute the cost matrix for the reference marker genes. First, import the locations and reference database.

In [35]:
import os
import pandas as pd
target_space_path = 'novosparc/datasets/bdtnp/geometry.txt'
locations = pd.read_csv(target_space_path, sep=' ')
num_locations = 3039
locations = locations[:num_locations][['xcoord', 'zcoord']].values

In [36]:
# bdtnp.txt is actually named dge.txt - this file only contains 3039 cells
bdtnp_path = 'novosparc/datasets/bdtnp/dge.txt'
insitu_genes = np.genfromtxt(bdtnp_path, usecols=range(84), dtype='str', max_rows=1)
insitu_matrix = np.loadtxt(bdtnp_path, usecols=range(84), skiprows=1)

In [ ]:
tissue_with_markers = novosparc.cm.Tissue(dataset=dataset.raw.to_adata(), locations=locations)

Then find the indices of the marker genes in the scRNAseq dataset

In [ ]:
gene_names = tissue_with_markers.gene_names
markers_in_sc = np.array([], dtype='int')
for marker in insitu_genes:
    marker_index = np.where(gene_names == marker)[0]
    if len(marker_index) > 0:
        markers_in_sc = np.append(markers_in_sc, marker_index[0])

In [ ]:
tissue_with_markers.setup_reconstruction(markers_to_use=markers_in_sc, insitu_matrix=insitu_matrix)

Choose a different value for `alpha_linear`. Here we choose `0.5`, meaning the geometric and gen marker components are equally weighted for the reconstruction.

In [ ]:
tissue_with_markers.reconstruct(alpha_linear=0.5, epsilon=5e-3)

Compute the sDGE and plot spatial gene expression patterns

In [ ]:
sdge = tissue_with_markers.sdge
print ('dimensions of the sDGE (genes, locations):', sdge.shape)

In [ ]:
plt.figure(figsize=(12,4))
plt.subplot(231)
plt.scatter(locations[:, 0], locations[:, 1], c=sdge[np.argwhere(gene_names == 'ftz')[0], :].flatten(), s=1)
plt.axis('off')
plt.subplot(232)
plt.scatter(locations[:, 0], locations[:, 1], c=sdge[np.argwhere(gene_names == 'twi')[0], :].flatten(), s=1)
plt.axis('off')
plt.subplot(233)
plt.scatter(locations[:, 0], locations[:, 1], c=sdge[np.argwhere(gene_names == 'zen')[0], :].flatten(), s=1)
plt.axis('off')
plt.subplot(234)
plt.scatter(locations[:, 0], locations[:, 1], c=sdge[np.argwhere(gene_names == 'ImpE2')[0], :].flatten(), s=1)
plt.axis('off')
plt.subplot(235)
plt.scatter(locations[:, 0], locations[:, 1], c=sdge[np.argwhere(gene_names == 'sim')[0], :].flatten(), s=1)
plt.axis('off')
plt.subplot(236)
plt.scatter(locations[:, 0], locations[:, 1], c=sdge[np.argwhere(gene_names == 'pyr')[0], :].flatten(), s=1)
plt.axis('off')
plt.show()
plt.close()